In [1]:
import math
import time, os
from itertools import chain
from tqdm import tqdm
import gc
import numpy as np
import torch
import psutil
import glob
import out_manager as om
import torch.nn.functional as F
from scipy.sparse.csgraph import shortest_path
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, GCNConv, SortAggregation
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix
from config import Config

In [2]:
config = Config()
# out_dir = om.get_existing_out_dir(config)
# om.setup_logging(os.path.join(out_dir, "seal_log.txt"))
seed = config.seed
torch.manual_seed(seed)
np.random.seed(seed)
device = config.device

In [3]:
class SEALDataset(InMemoryDataset):
    def __init__(self, dataset, num_hops, split='train'):
        self._data = dataset[0]
        self.num_hops = num_hops
        super().__init__(dataset.root)
        index = ['train', 'val', 'test'].index(split)
        self.load(self.processed_paths[index])

    @property
    def processed_dir(self):
        return os.path.join(self.root, 'Cora', 'split')
    
    @property
    def processed_file_names(self):
        return ['SEAL_train_data.pt', 'SEAL_val_data.pt', 'SEAL_test_data.pt']

    def process(self):
        transform = RandomLinkSplit(num_val=0.05, num_test=0.1,
                                    is_undirected=True, split_labels=True)
        train_data, val_data, test_data = transform(self._data)

        self._max_z = 0

        # Collect a list of subgraphs for training, validation and testing:
        train_pos_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.pos_edge_label_index, 1)
        train_neg_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.neg_edge_label_index, 0)

        val_pos_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.pos_edge_label_index, 1)
        val_neg_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.neg_edge_label_index, 0)

        test_pos_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.pos_edge_label_index, 1)
        test_neg_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.neg_edge_label_index, 0)

        # Convert node labeling to one-hot features.
        for data in chain(train_pos_data_list, train_neg_data_list,
                          val_pos_data_list, val_neg_data_list,
                          test_pos_data_list, test_neg_data_list):
            # We solely learn links from structure, dropping any node features:
            data.x = F.one_hot(data.z, self._max_z + 1).to(torch.float)
        print(self._max_z)

        train_data_list = train_pos_data_list + train_neg_data_list
        self.save(train_data_list, self.processed_paths[0])
        val_data_list = val_pos_data_list + val_neg_data_list
        self.save(val_data_list, self.processed_paths[1])
        test_data_list = test_pos_data_list + test_neg_data_list
        self.save(test_data_list, self.processed_paths[2])

    def extract_enclosing_subgraphs(self, edge_index, edge_label_index, y):
        data_list = []
        for src, dst in edge_label_index.t().tolist():
            try:
                # 直接采样子图（内部会重新编号）
                sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                    [src, dst], self.num_hops, edge_index, relabel_nodes=True)
                # 现在 src、dst 的新索引
                src_sub, dst_sub = mapping.tolist()

                # Remove target link from the subgraph
                mask1 = (sub_edge_index[0] != src_sub) | (sub_edge_index[1] != dst_sub)
                mask2 = (sub_edge_index[0] != dst_sub) | (sub_edge_index[1] != src_sub)
                sub_edge_index = sub_edge_index[:, mask1 & mask2]

                # 生成标签 (用子图索引！)
                z = self.drnl_node_labeling(sub_edge_index, src_sub, dst_sub,
                                            num_nodes=sub_nodes.size(0))

                # 直接用 sub_nodes 对应原图的 x 特征
                sub_data = Data(x=self._data.x[sub_nodes], z=z,
                            edge_index=sub_edge_index, y=y)
                data_list.append(sub_data)

            except Exception as e:
                print(f"Error at src {src}, dst {dst}: {e}")

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        # Double-radius node labeling (DRNL).
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        # Remove src
        idx_wo_src = list(range(adj.shape[0]))
        idx_wo_src.pop(src)
        adj_wo_src = adj[idx_wo_src, :][:, idx_wo_src]

        # Remove dst
        idx_wo_dst = list(range(adj.shape[0]))
        idx_wo_dst.pop(dst)
        adj_wo_dst = adj[idx_wo_dst, :][:, idx_wo_dst]

        # 重新计算 src/dst 在 adj_wo_* 中的索引
        src_idx_in_wo_dst = src if src < dst else src - 1
        dst_idx_in_wo_src = dst if dst < src else dst - 1

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src_idx_in_wo_dst)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst_idx_in_wo_src)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self._max_z = max(int(z.max()), self._max_z)
        return z.to(torch.long)

In [4]:
class SEAL():
    def __init__(self, num_hops, save_dir):
        self.save_dir = save_dir
        self.num_hops = num_hops
        super().__init__()

    # def get_max_z(self, data, edge_index, edge_label_index, y, batch_size=1000):
    #     for i in range(0, edge_label_index.size(1), batch_size):
    #         batch_idx = edge_label_index[:, i:i+batch_size]
    #         batch_data_list = self.extract_enclosing_subgraphs(data, edge_index, batch_idx, y)
    #         del batch_data_list
    #         gc.collect()

    # def save_batches(self, data, edge_index, edge_label_index, y, out_prefix, max_z, batch_size=100):
    #     os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
    #     idx = 0
    #     for i in range(0, edge_label_index.size(1), batch_size):
    #         batch_idx = edge_label_index[:, i:i+batch_size]
    #         batch_data_list = self.extract_enclosing_subgraphs(data, edge_index, batch_idx, y)
    #         for batch_data in batch_data_list:
    #             batch_data.x = F.one_hot(batch_data.z, max_z + 1).to(torch.float)
    #         torch.save(batch_data_list, f"{out_prefix}_batch{idx}.pt")
    #         print(f"已保存 {out_prefix}_batch{idx}.pt，当前内存占用率 {psutil.virtual_memory().percent}%")
    #         del batch_data_list
    #         gc.collect()
    #         idx += 1

    def get_max_z(self, data, edge_index, edge_label_index, y, batch_size=1000):
        num_samples = edge_label_index.size(1)
        # 使用tqdm添加进度条
        for i in tqdm(range(0, num_samples, batch_size), desc="扫描 max_z", unit="batch"):
            batch_idx = edge_label_index[:, i:i+batch_size]
            batch_data_list = self.extract_enclosing_subgraphs(data, edge_index, batch_idx, y)
            del batch_data_list
            gc.collect()

    def save_batches(self, data, edge_index, edge_label_index, y, out_prefix, max_z, batch_size=100):
        os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
        num_samples = edge_label_index.size(1)
        num_batches = (num_samples + batch_size - 1) // batch_size
        idx = 0
        # 使用tqdm添加进度条
        for i in tqdm(range(0, num_samples, batch_size), desc=f"保存 {out_prefix} 分批文件", unit="batch"):
            batch_idx = edge_label_index[:, i:i+batch_size]
            batch_data_list = self.extract_enclosing_subgraphs(data, edge_index, batch_idx, y)
            for batch_data in batch_data_list:
                batch_data.x = F.one_hot(batch_data.z, max_z + 1).to(torch.float)
            torch.save(batch_data_list, f"{out_prefix}_batch{idx}.pt")
            print(f"已保存 {out_prefix}_batch{idx}.pt，当前内存占用率 {psutil.virtual_memory().percent}%")
            del batch_data_list
            gc.collect()
            idx += 1

    def merge_batches(self, batch_prefix, out_file):
        batch_files = sorted(glob.glob(f"{batch_prefix}_batch*.pt"),
                            key=lambda x: int(x.split('_batch')[-1].split('.pt')[0]))
        all_data = []
        for batch_file in batch_files:
            data_list = torch.load(batch_file)
            all_data.extend(data_list)
            print(f"合并了 {batch_file}，当前总量：{len(all_data)}")
            del data_list
        torch.save(all_data, out_file)
        print(f"保存到 {out_file}，总计 {len(all_data)} 条数据")
        del all_data

    def merge_pos_neg(self, pos_file, neg_file, out_file):
        pos_data = torch.load(pos_file)
        neg_data = torch.load(neg_file)
        all_data = pos_data + neg_data
        torch.save(all_data, out_file)
        print(f"最终合并 {out_file}，总计 {len(all_data)} 条（正例 {len(pos_data)}，负例 {len(neg_data)}）")
        del pos_data, neg_data, all_data

    def process(self):

        train_data = torch.load(f'./data/{config.dataset}/split/train_data.pt')
        val_data = torch.load(f'./data/{config.dataset}/split/val_data.pt')
        test_data = torch.load(f'./data/{config.dataset}/split/test_data.pt')
        
        #第一次扫描统计maxz
        self._max_z = 0
        self.get_max_z(train_data, train_data.edge_index, train_data.pos_edge_label_index, 1),
        self.get_max_z(train_data, train_data.edge_index, train_data.neg_edge_label_index, 0)
        self.get_max_z(val_data, val_data.edge_index, val_data.pos_edge_label_index, 1),
        self.get_max_z(val_data, val_data.edge_index, val_data.neg_edge_label_index, 0)
        self.get_max_z(test_data, test_data.edge_index, test_data.pos_edge_label_index, 1),
        self.get_max_z(test_data, test_data.edge_index, test_data.neg_edge_label_index, 0)


        # 2. 分批次one-hot和保存，绝不汇总到内存
        save_dir = self.save_dir
        train_pos_path = os.path.join(save_dir, "SEAL_train_pos")
        train_neg_path = os.path.join(save_dir, "SEAL_train_neg")
        val_pos_path = os.path.join(save_dir, "SEAL_val_pos")
        val_neg_path = os.path.join(save_dir, "SEAL_val_neg")
        test_pos_path = os.path.join(save_dir, "SEAL_test_pos")
        test_neg_path = os.path.join(save_dir, "SEAL_test_neg")

        print("保存 train 分批文件")
        self.save_batches(train_data, train_data.edge_index, train_data.pos_edge_label_index, 1, train_pos_path, self._max_z)
        self.save_batches(train_data, train_data.edge_index, train_data.neg_edge_label_index, 0, train_neg_path, self._max_z)

        print("保存 val 分批文件")
        self.save_batches(val_data, val_data.edge_index, val_data.pos_edge_label_index, 1, val_pos_path, self._max_z)
        self.save_batches(val_data, val_data.edge_index, val_data.neg_edge_label_index, 0, val_neg_path, self._max_z)

        print("保存 test 分批文件")
        self.save_batches(test_data, test_data.edge_index, test_data.pos_edge_label_index, 1, test_pos_path, self._max_z)
        self.save_batches(test_data, test_data.edge_index, test_data.neg_edge_label_index, 0, test_neg_path, self._max_z)

        print("所有分批处理和保存已完成！🚀")

        del train_data, val_data, test_data
        gc.collect()

        # 2. 合并
        split_dir = self.save_dir
        for prefix in ["train", "val", "test"]:
            pos_prefix = os.path.join(split_dir, f"SEAL_{prefix}_pos")
            neg_prefix = os.path.join(split_dir, f"SEAL_{prefix}_neg")
            pos_data_list = pos_prefix + "_data_list.pt"
            neg_data_list = neg_prefix + "_data_list.pt"
            merged_data_list = os.path.join(split_dir, f"SEAL_{prefix}_data_list.pt")

            print(f"\n--- 合并 {prefix} pos batch ---")
            self.merge_batches(pos_prefix, pos_data_list)
            print(f"--- 合并 {prefix} neg batch ---")
            self.merge_batches(neg_prefix, neg_data_list)

            print(f"--- 合并 {prefix} pos+neg 为总 data_list ---")
            self.merge_pos_neg(pos_data_list, neg_data_list, merged_data_list)

        print("所有分批处理、合并已完成！🚀")

        # --------- 自动删除所有 batch 文件 ----------
        pattern = os.path.join(self.save_dir, "SEAL_*_batch*.pt")
        batch_files = glob.glob(pattern)
        for file in batch_files:
            try:
                os.remove(file)
                print(f"已删除 {file}")
            except Exception as e:
                print(f"删除 {file} 失败：{e}")
        
        target_files = [
            "SEAL_test_neg_data_list.pt",
            "SEAL_test_pos_data_list.pt",
            "SEAL_val_neg_data_list.pt",
            "SEAL_val_pos_data_list.pt",
            "SEAL_train_neg_data_list.pt",
            "SEAL_train_pos_data_list.pt"
        ]

        for filename in target_files:
            file_path = os.path.join(self.save_dir, filename)
            if os.path.exists(file_path):
                try:
                    os.remove(file_path)
                    print(f"已删除 {file_path}")
                except Exception as e:
                    print(f"删除 {file_path} 失败：{e}")

        print('所有数据已保存并清理临时 batch 文件')

    def extract_enclosing_subgraphs(self, data, edge_index, edge_label_index, y):
        data_list = []
        for src, dst in edge_label_index.t().tolist():
            # 直接采样子图（内部会重新编号）
            sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                [src, dst], self.num_hops, edge_index, relabel_nodes=True)
            # 现在 src、dst 的新索引
            src_sub, dst_sub = mapping.tolist()

            # Remove target link from the subgraph
            mask1 = (sub_edge_index[0] != src_sub) | (sub_edge_index[1] != dst_sub)
            mask2 = (sub_edge_index[0] != dst_sub) | (sub_edge_index[1] != src_sub)
            sub_edge_index = sub_edge_index[:, mask1 & mask2]

            # 生成标签 (用子图索引！)
            z = self.drnl_node_labeling(sub_edge_index, src_sub, dst_sub,
                                        num_nodes=sub_nodes.size(0))

            # 直接用 sub_nodes 对应原图的 x 特征
            sub_data = Data(x=data.x[sub_nodes], z=z,
                        edge_index=sub_edge_index, y=y)
            data_list.append(sub_data)

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                 indices=src)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                 indices=dst - 1)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self._max_z = max(int(z.max()), self._max_z)

        return z.to(torch.long)


In [5]:
seal = SEAL(num_hops = 2, save_dir = f'./data/{config.dataset}/split')
seal.process()

train_data_list = torch.load(f'./data/{config.dataset}/split/SEAL_train_data_list.pt')
val_data_list = torch.load(f'./data/{config.dataset}/split/SEAL_val_data_list.pt')
test_data_list = torch.load(f'./data/{config.dataset}/split/SEAL_test_data_list.pt')

train_loader = DataLoader(train_data_list, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=32)
test_loader = DataLoader(test_data_list, batch_size=32)


扫描 max_z: 100%|██████████| 5/5 [00:21<00:00,  4.34s/batch]


保存 train 分批文件


保存 ./data/PubMed/split\SEAL_train_pos 分批文件:   0%|          | 1/377 [00:00<04:28,  1.40batch/s]

已保存 ./data/PubMed/split\SEAL_train_pos_batch0.pt，当前内存占用率 41.6%


保存 ./data/PubMed/split\SEAL_train_pos 分批文件:   1%|          | 2/377 [00:01<04:49,  1.30batch/s]

...

保存 ./data/PubMed/split\SEAL_test_neg 分批文件:  98%|█████████▊| 44/45 [00:27<00:00,  1.60batch/s]

已保存 ./data/PubMed/split\SEAL_test_neg_batch43.pt，当前内存占用率 44.0%


保存 ./data/PubMed/split\SEAL_test_neg 分批文件: 100%|██████████| 45/45 [00:27<00:00,  1.61batch/s]

已保存 ./data/PubMed/split\SEAL_test_neg_batch44.pt，当前内存占用率 43.9%
所有分批处理和保存已完成！🚀

--- 合并 train pos batch ---


合并了 ./data/PubMed/split\SEAL_train_pos_batch0.pt，当前总量：100
合并了 ./data/PubMed/split\SEAL_train_pos_batch1.pt，当前总量：200
合并了 ./data/PubMed/split\SEAL_train_pos_batch2.pt，当前总量：300
合并了 ./data/PubMed/split\SEAL_train_pos_batch3.pt，当前总量：400
合并了 ./data/PubMed/split\SEAL_train_pos_batch4.pt，当前总量：500
合并了 ./data/PubMed/split\SEAL_train_pos_batch5.pt，当前总量：600
合并了 ./data/PubMed/split\SEAL_train_pos_batch6.pt，当前总量：700
合并了 ./data/PubMed/split\SEAL_train_pos_batch7.pt，当前总量：800
合并了 ./data/PubMed/split\SEAL_train_pos_batch8.pt，当前总量：900
合并了 ./data/PubMed/split\SEAL_train_pos_batch9.pt，当前总量：1000
合并了 ./data/PubMed/split\SEAL_train_pos_batch10.pt，当前总量：1100
合并了 ./data/PubMed/split\SEAL_train_pos_batch11.pt，当前总量：1200
合并了 ./data/PubMed/split\SEAL_train_pos_batch12.pt，当前总量：1300
合并了 ./data/PubMed/split\SEAL_train_pos_batch13.pt，当前总量：1400
合并了 ./data/PubMed/split\SEAL_train_pos_batch14.pt，当前总量：1500
合并了 ./data/PubMed/split\SEAL_train_pos_batch15.pt，当前总量：1600
合并了 ./data/PubMed/split\SEAL_train_pos_batch16.pt，当前总量：1700

In [6]:
# dataset = Planetoid('./data', name='Cora')

# train_dataset = SEALDataset(dataset, num_hops=2, split='train')
# val_dataset = SEALDataset(dataset, num_hops=2, split='val')
# test_dataset = SEALDataset(dataset, num_hops=2, split='test')

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)
# test_loader = DataLoader(test_dataset, batch_size=32)



In [9]:
class DGCNN(torch.nn.Module):
    def __init__(self, hidden_dim, num_layers, GNN=GCNConv, k=0.6):
        super().__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_data_list])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = int(max(10, k))

        self.convs = ModuleList()
        self.convs.append(GNN(train_data_list[0].x.size(1), hidden_dim))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_dim, hidden_dim))
        self.convs.append(GNN(hidden_dim, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_dim * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.pool = SortAggregation(k)
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, norm=None)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = self.pool(x, batch)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = self.conv1(x).relu()
        x = self.maxpool1d(x)
        x = self.conv2(x).relu()
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        return self.mlp(x)


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(hidden_dim=32, num_layers=3).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()

In [11]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_data_list)

In [12]:
@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred)), average_precision_score(torch.cat(y_true),torch.cat(y_pred) )

In [13]:
times = []
best_val_auc = test_auc = 0
for epoch in range(1, 51):
    start = time.time()
    loss = train()
    val_auc, val_ap = test(val_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_auc, test_ap = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val_AUC: {val_auc:.4f}, Val_AP: {val_ap:.4f}'
          f'Test_AUC: {test_auc:.4f}, Test_AP: {test_ap:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 01, Loss: 0.4460, Val_AUC: 0.9308, Val_AP: 0.9307Test_AUC: 0.9336, Test_AP: 0.9347
Epoch: 02, Loss: 0.3235, Val_AUC: 0.9432, Val_AP: 0.9448Test_AUC: 0.9463, Test_AP: 0.9482
Epoch: 03, Loss: 0.3023, Val_AUC: 0.9473, Val_AP: 0.9484Test_AUC: 0.9492, Test_AP: 0.9504
Epoch: 04, Loss: 0.2951, Val_AUC: 0.9486, Val_AP: 0.9492Test_AUC: 0.9496, Test_AP: 0.9510
Epoch: 05, Loss: 0.2896, Val_AUC: 0.9508, Val_AP: 0.9513Test_AUC: 0.9529, Test_AP: 0.9539
Epoch: 06, Loss: 0.2855, Val_AUC: 0.9519, Val_AP: 0.9525Test_AUC: 0.9538, Test_AP: 0.9546
Epoch: 07, Loss: 0.2830, Val_AUC: 0.9535, Val_AP: 0.9537Test_AUC: 0.9565, Test_AP: 0.9571
Epoch: 08, Loss: 0.2792, Val_AUC: 0.9532, Val_AP: 0.9537Test_AUC: 0.9565, Test_AP: 0.9571
Epoch: 09, Loss: 0.2752, Val_AUC: 0.9555, Val_AP: 0.9557Test_AUC: 0.9587, Test_AP: 0.9591
Epoch: 10, Loss: 0.2716, Val_AUC: 0.9555, Val_AP: 0.9552Test_AUC: 0.9587, Test_AP: 0.9591
Epoch: 11, Loss: 0.2684, Val_AUC: 0.9554, Val_AP: 0.9554Test_AUC: 0.9587, Test_AP: 0.9591
Epoch: 12,